# Lab 10: Enzyme Kinetics (Part 2)

This is the *second* portion of the code to analyze the data for lab 10. This portion contains:
- D: Fitting the Michaelis-Menten curve to the data
- E: Fitting the Lineweaver-Burk equation to the data

**Please run all cells sequentially.**

I have indicated all points where you need to interact/change the code with the heading below:

## Task #0

There are **eight** tasks in total for *this notebook*. **Note:** Task #4 has two parts.

As always, make sure to read the instructions carefully.

Start by running the code below to get all the files necessary.

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/samihat-rahman/chempython-visions-2024.git

## Part D: Michaelis-Menten Fit

## Task #1
### Upload the data for the kinetic trials
Make sure all your kinetic data is stored in the Excel file named **"enzyme_kinetics_data.xlsx"**, regardless of whether you used python or excel to extrapolate your rates of reaction.

**DO NOT CHANGE ANY OF THE COLUMN HEADINGS. THIS WILL RESULT IN AN ERROR IN THE CODE** 

The code block below does not need to be changed, but make sure to run it.

In [ ]:
# Import the libraries so we can run this independently
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# We'll load the data from the file
kinetics_data = pd.read_excel("enzyme_kinetics_data.xlsx", engine="openpyxl")

# Convert to arrays
concentration = kinetics_data["Concentration"].to_numpy()
initial_rate = kinetics_data["Rate"].to_numpy()

# Test to see if the data is loaded correctly
print(f"Concentration: {concentration}")
print(f"Initial Rate: {initial_rate}")

The Michaelis-Menten equation is:
$$v = \frac{V_{max}[S]}{K_m + [S]}$$
where  
- $v$ is the rate of reaction
- $V_{max}$ is the maximum rate of reaction
- $[S]$ is the substrate concentration
- $K_m$ is the Michaelis constant


#### Using lmfit

We used lmfit in the previous lab but since it is advanced, we will walk through most of the steps again.

## Task #2

We will need to first write a function that contains the equation we will be fitting to our data, in this case it's the Michaelis-Menten equation, which I will label `michaelis_menten`. I've added the input variables and the output variable of the function, I will have you add the equation using the syntax you have learned so far.


Here are the definitions of all the variables I used:
- `s` = substrate concentration
- `v_max` = maximum rate of reaction
- `k_m` = Michaelis constant
- `v` = rate of reaction

In [ ]:
from lmfit import Model

# This is this Michaelis-Menten Equation to explain our kinetics
def michaelis_menten(s, v_max, k_m):
  # Write the equation below using the variables above and below
  
  return v

#### Creating a fit model

As a recap from lab 9:

We will create a variable for the Model of the fit, where we basically tell lmfit what function we want to fit and what our independent variable is. We then have to create the parameters of the fit, i.e. what values we are trying to obtain by fitting this function ($V_{max}$ and $K_m$). Then, we have to add some guesses as to what we think the values *might* be, these guesses do not have to be close, just a starting point for the model.

## Task #3
Replace the question mark with the appropriate independent variable.

In [ ]:
# Fitting
# This is the model, i.e. function we will fit
# Notice that there is an independent variable defined here, this is the same as the experimental independent variable
# add the variable name for the independent variable in place of the ? make sure to keep the quotes
m_m_fit = Model(michaelis_menten, independent_vars='?')
# These are the two unknown parameters we want to fit, and their "guesses" which are after the "=" signs
pars = m_m_fit.make_params(k_m=20, v_max=20)

#### Fit the data!
Same as before but as a reminder this is what each argument means:
- `data` = the experimental "y" variables, i.e. the rate of reaction
- `params` = the guesses for the parameters (our `pars` variable)
- `method` = what fitting method we're using, in this course will only use `'leastsq'`
- `s` = the independent variable (**Note:** this will change depending on what we define as our independent variable)
- `scale_covar` = this makes sure the errors obtained are scaled correctly

The total fit is stored in a variable labeled `result`.
After fitting, we'll extrapolate the fitted slope, its error, and the $r^2$ value.

In [ ]:
# Finally, we start fitting, where this function will try our different values until it gets the closest fit
result = m_m_fit.fit(data=initial_rate, params=pars, method='leastsq', s=concentration, scale_covar=True, nan_policy='propagate')

# We then extrapolate the fitted parameters
vmax_fit = result.params['v_max'].value
km_fit = result.params['k_m'].value

# And the errors from the fit as well from something known as the covariance matrix
vmax_err, km_err = np.sqrt(np.diag(result.covar))

# We can also see how good the fit was
r_squared = 1 - result.residual.var() / np.var(initial_rate)

# Finally, print the results
print(f'vmax = {vmax_fit} ± {vmax_err}')
print(f'km = {km_fit} ± {km_err}')
print(f'r^2 = {r_squared}')

#### Making the plot
This part should be familiar by now--this is the exact same plotting procedure as when we did linear regressions. Except, now that we have the Michaelis-Menten function, we can calculate the `fitted_v` by calling the function and using our fitted parameters.

## Task #4

### Task #4.1
Below the section headed `'''CHANGE AXIS LABELS'''`, there are two commands labeling the x-axis (`plt.xlabel`) and y-axis (`plt.ylabel`). The actual labels are in-between quotation marks, and inside the quotes I've added a question mark in place of where the **units** for the axes should be. Replace this question mark only with the appropriate units.

### Task #4.2

Add the data to the plot. Below the section headed `'''PLOTTING THE DATA'''`, replace the `x` and `y` with the appropriate variables to plot *both* your experimental data and the fitted data.

In [ ]:
# to make a smooth plot, we'll create some equally distributed concentration points
fit_concentration = np.linspace(start=0, stop=1.3, num=200)
# now we can calulate the fitted rates using the parameters from the fit
fitted_v = michaelis_menten(fit_concentration, vmax_fit, km_fit)

# Plotting
# Create a canvas for the figure thats 8pt wide and 6pt tall
plt.figure(figsize=(8, 6))

# This changes the fontsize of the ticks on the axis
plt.tick_params(labelsize=14)

'''CHANGE AXIS LABELS'''
# This labels the axis and also changes their fontsize
# Replace the ? with the appropriate units for the x and y axis
plt.xlabel('Concentration (?)', fontsize=16)
plt.ylabel('Rate (?)', fontsize=16)

# This changes the limits of the x and y axis
plt.xlim(0, 1.2)
plt.ylim(0, 0.00065)

'''PLOTTING THE DATA'''
# Plotting the experimental data as a scatter plot hence the "o"
# Replace the x and y with the appropriate variables
plt.plot(x, y, 'o', markersize=8, color='red', label='Experimental Data')
# The fitted data is plotted as a smooth line
# Replace the x and y with the appropriate variables
plt.plot(x, y, color='black', label='Michaelis-Menten Fit')

# add text to the plot in the top left corner
plt.text(0.05, 0.95, 
         f'V$_\\mathrm{{max}}$ = {vmax_fit:0.3g} ± {vmax_err:0.1g}' + 
         f'\nK$_\\mathrm{{m}}$= {km_fit:0.3g} ± {km_err:0.1g}' + 
         f'\n$r^2$ = {r_squared:0.3g}',
         ha='left', va='top', transform=plt.gca().transAxes, fontsize=16)

# a legend to help identify our plots
plt.legend(fontsize=16)

# this makes sure that the plot is formatted correctly
plt.tight_layout()

## Part E: Lineweaver-Burk Fit
When we take the reciprocal of the Michaelis-Menten equation we get the Lineweaver-Burk equation:
$$\frac{1}{v} = \frac{K_m}{V_{max}[S]} + \frac{1}{V_{max}}$$

This shows a linear relationship between $\frac{1}{v}$ and $\frac{1}{[S]}$, so we can extrpolate $V_{max}$ and $K_m$ by doing some algebra.
We'll use `linregress` again to do the fit, so I will skip the steps as it is the same as before.

## Task #5
I've added two *incomplete variables* for the reciprocal of the concentration and the initial rate. Complete the variables with the correct notation needed to find $\frac{1}{[S]}$ and $\frac{1}{v}$.

In [ ]:
# Finding the reciprocals
# Compute the reciprocal of the concentration and the initial rate
# The variables for the concentration and initial rate are the same as before
concentration_reciprocal = 
initial_rate_reciprocal = 

## Task #6
Add the x and y variables that we want to fit for this regression (after the equals signs)

In [ ]:
# Same as before, we'll use linregress to get the slope and intercept
from scipy.stats import linregress

regression_results = linregress(x=, y=)

# Here's the fitted slope and intercept
slope = regression_results.slope
intercept = regression_results.intercept

# And the errors from the fit as well
slope_err = regression_results.stderr
intercept_err = regression_results.intercept_stderr

# Find the r^2
r_squared = regression_results.rvalue**2

# Printing the results
print(f'Slope = {slope} ± {slope_err}')
print(f'Intercept = {intercept} ± {intercept_err}')
print(f'r^2 = {r_squared}')

Comparing the Lineweaver-Burk equation to the linear equation, we can extrapolate $V_{max}$ and $K_m$.

## Task #7
I've calculated the fitted $V_{max}$ for you already. In similar fashion, complete the variable for `k_m_fit=` with the appropriate operation needed to extrapolate $K_m$ from your fitted parameters above.

In [ ]:
# Now to find Vmax and Km from the fit parameters and their errors
v_max_fit = 1/intercept
v_max_err = 1/intercept_err

# Complete the equation to find Km from the fitted parameters and Vmax
k_m_fit = 

# This is using error propagation: you don't need to know this
k_m_err = ((slope_err/slope) + (v_max_err/v_max_fit)) * k_m_fit

print(f'vmax = {vmax_fit} ± {vmax_err}')
print(f'km = {km_fit} ± {km_err}')
print(f'r^2 = {r_squared}')

The plotting procedure is also the same as before.

## Task #8
Add the data to the plot. Below the section headed `'''PLOTTING THE DATA'''`, replace the `x` and `y` with the appropriate variables to plot *both* your experimental data and the fitted data.

In [ ]:
# To make a smooth plot, we'll create some equally distributed concentration points
# We're skipping the first point since it's 0
fit_concentration_reciprocal = 1/fit_concentration[1:]
# now we can calulate the fitted rates using the parameters from the fit
fitted_v_reciprocal = fit_concentration_reciprocal * slope + intercept

# Plotting
# Create a canvas for the figure thats 8pt wide and 6pt tall
plt.figure(figsize=(8, 6))

# This changes the fontsize of the ticks on the axis
plt.tick_params(labelsize=14)

# This labels the axis and also changes their fontsize
plt.xlabel('$1/[S]$', fontsize=16)
plt.ylabel('$1/v$', fontsize=16)

# This changes the limits of the x and y axis
plt.xlim(0, 40)
plt.ylim(1000, 30000)

'''PLOTTING THE DATA'''
# plotting the experimental data as a scatter plot hence the "o"
# Replace the x and y with the appropriate variables
plt.plot(x, y, 'o', markersize=8, color='red', label='Experimental Data')
# the fitted data is plotted as a smooth line
# Replace the x and y with the appropriate variables
plt.plot(x, y, color='black', label='Lineweaver-Burk Fit')

# add text to the plot in the top left corner
plt.text(0.05, 0.95, 
         f'V$_\\mathrm{{max}}$ = {vmax_fit:0.3g} ± {vmax_err:0.1g}' + 
         f'\nK$_\\mathrm{{m}}$= {km_fit:0.3g} ± {km_err:0.1g}' + 
         f'\n$r^2$ = {r_squared:0.3g}',
         ha='left', va='top', transform=plt.gca().transAxes, fontsize=16)

# a legend to help identify our plots
plt.legend(fontsize=16)

# this makes sure that the plot is formatted correctly
plt.tight_layout()